In [1]:
!pip install bs4
!conda install -c conda-forge folium=0.5.0

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=f1dc057391762d714238eeb494c482c1f2dfe15275b61475cb0aca4ad98d2272
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1k            

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import numpy as np

In [3]:
url="https://www.geonames.org/postalcode-search.html?q=&country=AU&adminCode1=VIC"
html_data=requests.get(url).text

In [4]:
soup=BeautifulSoup(html_data,"html5lib")

In [5]:
mel_table=soup.find_all("table",class_='restable')
mel_table

[<table class="restable">
 <tbody><tr><th></th><th>Place</th><th>Code</th><th>Country</th><th>Admin1</th><th>Admin2</th><th>Admin3</th></tr>
 <tr><td><small>1</small></td><td>Melbourne</td><td>3000</td><td>Australia</td><td>Victoria</td><td>MELBOURNE CITY</td><td></td></tr><tr><td></td><td colspan="6">   <a href="/maps/browse_-37.813_144.961.html" rel="nofollow"><small>-37.813/144.961</small></a></td></tr>
 <tr class="odd"><td><small>2</small></td><td>East Melbourne</td><td>3002</td><td>Australia</td><td>Victoria</td><td>MELBOURNE CITY</td><td></td></tr><tr class="odd"><td></td><td colspan="6">   <a href="/maps/browse_-37.813_144.984.html" rel="nofollow"><small>-37.813/144.984</small></a></td></tr>
 <tr><td><small>3</small></td><td>West Melbourne</td><td>3003</td><td>Australia</td><td>Victoria</td><td>MELBOURNE CITY</td><td></td></tr><tr><td></td><td colspan="6">   <a href="/maps/browse_-37.809_144.947.html" rel="nofollow"><small>-37.809/144.947</small></a></td></tr>
 <tr class="odd"><

In [6]:
table_rows=mel_table[0].find_all("tr")
l=[]
for tr in table_rows:
    col=tr.find_all("td")
    row=[tr.text for tr in col]
    l.append(row)
    
all_data=pd.DataFrame(l,columns=["index","Suburb","Code","Country","State","City","None"])
all_data

,index,Suburb,Code,Country,State,City,None
0,None,None,None,None,None,None,None
1,1,Melbourne,3000,Australia,Victoria,MELBOURNE CITY,
2,,-37.813/144.961,None,None,None,None,None
3,2,East Melbourne,3002,Australia,Victoria,MELBOURNE CITY,
4,,-37.813/144.984,None,None,None,None,None
...,...,...,...,...,...,...,...
397,199,Hillside,3037,Australia,Victoria,FOOTSCRAY,
398,,-37.691/144.743,None,None,None,None,None
399,200,Taylors Hill,3037,Australia,Victoria,,
400,,-37.715/144.753,None,None,None,None,None


In [7]:
is_place=(all_data["index"]!="")&(all_data["index"]!=None)
place_data=all_data[is_place]
place_data2=place_data.dropna()
place_data2.reset_index(inplace=True)
place_data2=place_data2.drop(place_data2.columns[[0,1,3,7]],axis=1)
place_data2

,Suburb,Country,State,City
0,Melbourne,Australia,Victoria,MELBOURNE CITY
1,East Melbourne,Australia,Victoria,MELBOURNE CITY
2,West Melbourne,Australia,Victoria,MELBOURNE CITY
3,Melbourne,Australia,Victoria,MELBOURNE CITY
4,Docklands,Australia,Victoria,MELBOURNE CITY
...,...,...,...,...
195,Travancore,Australia,Victoria,FOOTSCRAY
196,Avondale Heights,Australia,Victoria,FOOTSCRAY
197,Sydenham,Australia,Victoria,FOOTSCRAY
198,Hillside,Australia,Victoria,FOOTSCRAY


In [8]:
index_coord=all_data.index[all_data.isnull().any(axis=1)]

In [9]:
is_coord=all_data.iloc[index_coord,:]
is_coord2=is_coord.drop([is_coord.index[0],is_coord.index[201]])
is_coord2=is_coord2.rename(columns={'Suburb':'Coordinates'})
is_coord2[['Latitude','Longitude']]=is_coord2['Coordinates'].str.split('/',expand=True)
is_coord2.reset_index(inplace=True)
is_coord2=is_coord2.drop(is_coord.columns[[0,2,3,4,5,6]],axis=1)
is_coord2

,level_0,Coordinates,Latitude,Longitude
0,2,-37.813/144.961,-37.813,144.961
1,4,-37.813/144.984,-37.813,144.984
2,6,-37.809/144.947,-37.809,144.947
3,8,-37.842/144.976,-37.842,144.976
4,10,-37.818/144.944,-37.818,144.944
...,...,...,...,...
195,392,-37.783/144.937,-37.783,144.937
196,394,-37.761/144.862,-37.761,144.862
197,396,-37.7/144.766,-37.7,144.766
198,398,-37.691/144.743,-37.691,144.743


In [10]:
combined_data=pd.concat([place_data2,is_coord2],axis=1)
combined_data=combined_data.drop(combined_data.columns[[4]],axis=1)
combined_data

,Suburb,Country,State,City,Coordinates,Latitude,Longitude
0,Melbourne,Australia,Victoria,MELBOURNE CITY,-37.813/144.961,-37.813,144.961
1,East Melbourne,Australia,Victoria,MELBOURNE CITY,-37.813/144.984,-37.813,144.984
2,West Melbourne,Australia,Victoria,MELBOURNE CITY,-37.809/144.947,-37.809,144.947
3,Melbourne,Australia,Victoria,MELBOURNE CITY,-37.842/144.976,-37.842,144.976
4,Docklands,Australia,Victoria,MELBOURNE CITY,-37.818/144.944,-37.818,144.944
...,...,...,...,...,...,...,...
195,Travancore,Australia,Victoria,FOOTSCRAY,-37.783/144.937,-37.783,144.937
196,Avondale Heights,Australia,Victoria,FOOTSCRAY,-37.761/144.862,-37.761,144.862
197,Sydenham,Australia,Victoria,FOOTSCRAY,-37.7/144.766,-37.7,144.766
198,Hillside,Australia,Victoria,FOOTSCRAY,-37.691/144.743,-37.691,144.743


In [11]:
city_coord=combined_data[['Suburb','City','Latitude','Longitude']].copy()
city_coord

,Suburb,City,Latitude,Longitude
0,Melbourne,MELBOURNE CITY,-37.813,144.961
1,East Melbourne,MELBOURNE CITY,-37.813,144.984
2,West Melbourne,MELBOURNE CITY,-37.809,144.947
3,Melbourne,MELBOURNE CITY,-37.842,144.976
4,Docklands,MELBOURNE CITY,-37.818,144.944
...,...,...,...,...
195,Travancore,FOOTSCRAY,-37.783,144.937
196,Avondale Heights,FOOTSCRAY,-37.761,144.862
197,Sydenham,FOOTSCRAY,-37.7,144.766
198,Hillside,FOOTSCRAY,-37.691,144.743


In [12]:
unique_cities=city_coord['City'].unique()
unique_cities

array(['MELBOURNE CITY', 'FOOTSCRAY', '', 'MELB NORTH WEST', 'PRESTON',
       'HAWTHORN', 'RINGWOOD', 'MOORABBIN', 'FERNTREE GULLY', 'MULGRAVE',
       'DANDENONG', 'FRANKSTON', 'VIC COUNTRY', 'VIC FAR COUNTRY'],
      dtype=object)

In [13]:
suburb_list=['Balwyn','Melbourne','Footscray','Tarneit','Glenroy','Doncaster','Richmond','Blackburn','Mount Waverly','Keysborough','Frankston','Belmont','Portland']
suburb_data = city_coord[city_coord['Suburb'].isin(suburb_list)]
suburb_data=suburb_data.drop(suburb_data.columns[[1]],axis=1)
suburb_data

,Suburb,Latitude,Longitude
0,Melbourne,-37.813,144.961
3,Melbourne,-37.842,144.976
5,Footscray,-37.798,144.895
20,Tarneit,-37.844,144.673
31,Glenroy,-37.704,144.924
56,Balwyn,-37.81,145.083
60,Doncaster,-37.785,145.125
62,Richmond,-37.819,145.001
68,Blackburn,-37.821,145.151
93,Keysborough,-37.996,145.169


In [14]:
data_frame_of_coordinates = pd.DataFrame(suburb_data, columns=["Suburb", "Latitude","Longitude"])
data_frame_of_coordinates

,Suburb,Latitude,Longitude
0,Melbourne,-37.813,144.961
3,Melbourne,-37.842,144.976
5,Footscray,-37.798,144.895
20,Tarneit,-37.844,144.673
31,Glenroy,-37.704,144.924
56,Balwyn,-37.81,145.083
60,Doncaster,-37.785,145.125
62,Richmond,-37.819,145.001
68,Blackburn,-37.821,145.151
93,Keysborough,-37.996,145.169


In [15]:
address = 'Melbourne City, AUS'

geolocator = Nominatim(user_agent="aus_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Melbourne City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne City are -37.8166793, 144.9924662.


In [16]:
#pull map for Melbourne City, Australia
map_aus = folium.Map(location=[latitude, longitude], zoom_start=12)
label = "Melbourne City, AUS"
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude, longitude],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_aus)
    
map_aus

In [28]:
CLIENT_ID = '24IEQLH1RRTKZ0BE20SB13XYEMLSER5PUBDLATNT45IAYXE2' # your Foursquare ID
CLIENT_SECRET = 'HBZFTL3D4RZ1W1IZZWJABUW0A0LTGCNROGDSWNACRLE00ONW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [21]:
venues = []
radius=200
query = ['yoga']
for latitude, longtitude, name in zip(data_frame_of_coordinates['Latitude'], data_frame_of_coordinates['Longitude'], data_frame_of_coordinates['Suburb']):
    url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={latitude},{longtitude}&query={query}&radius={radius}&limit={LIMIT}"
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((latitude, 
                       longtitude,
                       venue['venue']['name'],
                       venue['venue']['location']['lat'],
                       venue['venue']['location']['lng'],
                       venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [29]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
melbourne_venues = getNearbyVenues(names=suburb_data['Suburb'],
                                   latitudes=suburb_data['Latitude'],
                                   longitudes=suburb_data['Longitude']
                                  )

Melbourne


KeyError: 'groups'